In [3]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "xlm-roberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

c:\Users\simba\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\simba\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
xlmr_params = model.num_parameters()
print(f"XLM-R Base has {xlmr_params/1000000}M parameters ")

XLM-R Base has 278.295186M parameters 


In [18]:
text = "I am a <mask>"

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [19]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits

mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> Ben 10 has a secret that allows him to transform into aliens.'
'>>> Ben 10 has a system that allows him to transform into aliens.'
'>>> Ben 10 has a weapon that allows him to transform into aliens.'
'>>> Ben 10 has a skill that allows him to transform into aliens.'
'>>> Ben 10 has a technology that allows him to transform into aliens.'


In [36]:
from datasets import load_dataset, Dataset, DatasetDict, Features, Value
import os, csv
import pandas as pd

data_dir = "../data"
langs = ["NR","SS","XH","ZU"]
datasets = {}

for lang in langs:
    lang_set = {
        "TRAIN": Dataset.from_pandas(
            pd.read_csv(f"../data/TRAIN/{lang}_TRAIN.tsv", delimiter="\t", quoting=csv.QUOTE_NONE)
        ),
        "TEST": Dataset.from_pandas(
            pd.read_csv(f"../data/TEST/{lang}_TEST.tsv", delimiter="\t", quoting=csv.QUOTE_NONE)
        ),
    }

    datasets[lang] = DatasetDict(lang_set)

datasets

{'NR': DatasetDict({
     TRAIN: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 44663
     })
     TEST: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 5026
     })
 }),
 'SS': DatasetDict({
     TRAIN: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 42596
     })
     TEST: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 4789
     })
 }),
 'XH': DatasetDict({
     TRAIN: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 43825
     })
     TEST: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 4910
     })
 }),
 'ZU': DatasetDict({
     TRAIN: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 44142
     })
     TEST: Dataset({
         features: ['word', 'seg', 'parse'],
         num_rows: 4955
     })
 })}